<H1> Лбораторная работа 3</h2>


Загружаем библиотеки

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Подготавливаем данные: определяем признаки и таргет, разделяем признаки на категориальные и численные и приобразуем их для обучения


In [ ]:
df = pd.read_csv("fraud.csv",sep=',')

features = ['trans_date_trans_time', 'amt', 'category', 'merchant', 'city', 'state', 'job', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'gender']
target = 'is_fraud'

fraud=df[df[target]==1]
not_fraud=df[df[target]==0]
not_fraud_down_sample=not_fraud.sample(n=15000)
under=pd.concat([fraud,not_fraud_down_sample])

x = under[features]
y = under[target]

x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour
x=x.drop(['trans_date_trans_time'],axis=1)
categorical_features = ['category', 'merchant', 'city', 'state', 'job', 'gender']
numeric_features = ['amt', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'hour']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
x = preprocessor.fit_transform(x)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2)

Создадим модель линейной регрессии для прогнозирования и обучим ее

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

Применим модель логистической регрессии для прогнозирования и оценим ее качество

In [13]:
predictions = model.predict(x_test)
binary=[1 if pred>=0.5 else 0 for pred in predictions]
print(metrics.classification_report(y_test,binary,digits=5))

              precision    recall  f1-score   support

           0    0.94731   0.96200   0.95460     11999
           1    0.70216   0.62609   0.66195      1717

    accuracy                        0.91995     13716
   macro avg    0.82473   0.79404   0.80827     13716
weighted avg    0.91662   0.91995   0.91796     13716



In [14]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)
pred=model.predict(x_test)
print(metrics.classification_report(y_test,pred,digits=5))

              precision    recall  f1-score   support

           0    0.95011   0.99033   0.96980     11999
           1    0.90405   0.63658   0.74710      1717

    accuracy                        0.94605     13716
   macro avg    0.92708   0.81345   0.85845     13716
weighted avg    0.94434   0.94605   0.94192     13716

